# R_5.2 Join Electorate to sa1
- so we can take intersection and proportionate data

### Import and preprocess data into correct geographical form

In [1]:
import geopandas as gpd
from pyproj import CRS

/Applications/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
gdf = gpd.read_file('../data/raw/Geography/2021_ELB_region.shp')

In [3]:
# correct geocode for Australia
new_crs = CRS.from_epsg(4283)  

# Reproject the GeoDataFrame to the new CRS
gdf = gdf.to_crs(new_crs)

In [4]:
# remove redundant columns
gdf = gdf[['E_div_numb', 'Elect_div', 'Area_SqKm', 'geometry']]
gdf

,E_div_numb,Elect_div,Area_SqKm,geometry
0,1,Banks,49.47,"POLYGON Z ((151.12967 -33.97363 0.00000, 151.1..."
1,2,Barton,39.65,"POLYGON Z ((151.17424 -33.92497 0.00000, 151.1..."
2,3,Bennelong,58.76,"POLYGON Z ((151.15908 -33.79871 0.00000, 151.1..."
3,4,Berowra,741.64,"POLYGON Z ((151.28480 -33.57221 0.00000, 151.2..."
4,5,Blaxland,61.16,"POLYGON Z ((151.04441 -33.84205 0.00000, 151.0..."
...,...,...,...,...
146,11,O'Connor,1126936.75,"MULTIPOLYGON Z (((129.00205 -30.76778 0.00000,..."
147,12,Pearce,782.75,"POLYGON Z ((115.98284 -31.74189 0.00000, 115.9..."
148,13,Perth,79.92,"POLYGON Z ((115.96432 -31.88769 0.00000, 115.9..."
149,14,Swan,150.89,"POLYGON Z ((116.04102 -31.98350 0.00000, 116.0..."


In [5]:
gdf['own_calculated_area'] = gdf.geometry.area

/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_72321/3467881995.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['own_calculated_area'] = gdf.geometry.area


In [6]:
gdf2 = gpd.read_file('../data/raw/Geography/SA1_2021_AUST_GDA2020.shp')

In [7]:
# remove redundant columns
gdf2 = gdf2[['SA1_CODE21', 'GCC_CODE21', 'STE_CODE21', 'AREASQKM21', 'geometry']]

In [8]:
new_crs = CRS.from_epsg(4283)  # Example: WGS 84

# Reproject the GeoDataFrame to the new CRS
gdf2 = gdf2.to_crs(new_crs)

In [9]:
# calculate the intersection of the polygons
intersection = gpd.overlay(gdf, gdf2, how='intersection')

/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_72321/589083907.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 16555 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  intersection = gpd.overlay(gdf, gdf2, how='intersection')


In [10]:
# Reproject the GeoDataFrame to the new CRS
intersection = intersection.to_crs(new_crs)

In [11]:
# find the intersecting area
intersection['intersection_area'] = intersection.geometry.area

/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_72321/2089829042.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  intersection['intersection_area'] = intersection.geometry.area


In [12]:
# find the percentage of the overlapped area within the electorate
intersection['percentage_overlap'] = intersection['intersection_area']/intersection['own_calculated_area']

In [1]:
# EDA to see number of meaningful overlap (i.e. consider any below 0.01 as rounding errors, and equally above 0.99)
len(intersection[(intersection['percentage_overlap'] < 0.99) & (intersection['percentage_overlap'] >0.01)])

In [15]:
intersection.to_file('../data/curated/GeographicalProportionateData/Elect_SA1.shp')

/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_72321/4220631060.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  intersection.to_file('../data/curated/GeographicalProportionateData/Elect_SA1.shp')


In [16]:
intersection[['E_div_numb', 'Elect_div', 'SA1_CODE21', 'percentage_overlap']].to_csv('../data/curated/GeographicalProportionateData/Elect_SA1.csv')